
<center>
<table>
  <tr>
    <td><img src="http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://github.com/astg606/py_materials/blob/master/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>


---

<center><h1>
    <font color="red" size="+2">Running Python</font>  
</h1></center>

In [1]:
# Cell 1 - imports and dataset (synthetic Telco-like dataset)
import numpy as np
import pandas as pd
np.random.seed(42)

n = 3000
def rchoice(opts, p=None):
    return np.random.choice(opts, size=n, p=p)

gender = rchoice(["Female","Male"], p=[0.5,0.5])
SeniorCitizen = np.random.binomial(1, 0.16, size=n)
Partner = rchoice(["Yes","No"], p=[0.45,0.55])
Dependents = rchoice(["Yes","No"], p=[0.25,0.75])
tenure = np.random.randint(0, 73, size=n)
PhoneService = rchoice(["Yes","No"], p=[0.9,0.1])
MultipleLines = rchoice(["No phone service","No","Yes"], p=[0.1,0.6,0.3])
InternetService = rchoice(["DSL","Fiber optic","No"], p=[0.4,0.45,0.15])
OnlineSecurity = rchoice(["No internet service","No","Yes"], p=[0.15,0.55,0.3])
OnlineBackup = rchoice(["No internet service","No","Yes"], p=[0.15,0.6,0.25])
DeviceProtection = rchoice(["No internet service","No","Yes"], p=[0.15,0.6,0.25])
TechSupport = rchoice(["No internet service","No","Yes"], p=[0.15,0.6,0.25])
StreamingTV = rchoice(["No internet service","No","Yes"], p=[0.15,0.5,0.35])
StreamingMovies = rchoice(["No internet service","No","Yes"], p=[0.15,0.5,0.35])
Contract = rchoice(["Month-to-month","One year","Two year"], p=[0.55,0.25,0.2])
PaperlessBilling = rchoice(["Yes","No"], p=[0.6,0.4])
PaymentMethod = rchoice(["Electronic check","Mailed check","Bank transfer (automatic)","Credit card (automatic)"], p=[0.35,0.2,0.25,0.2])
MonthlyCharges = np.round(np.random.normal(70, 30, size=n).clip(18,200),2)
TotalCharges = np.round(MonthlyCharges * tenure + np.random.normal(0, 50, size=n),2)
# synthetic churn signal
churn_prob = (
    0.25*(Contract=="Month-to-month").astype(float) +
    0.15*(InternetService=="Fiber optic").astype(float) +
    0.12*(PaperlessBilling=="Yes").astype(float) +
    0.2*(tenure<12).astype(float) +
    0.05*(MonthlyCharges>100).astype(float) +
    np.random.normal(0, 0.05, size=n)
)
churn_prob = np.clip(churn_prob, 0.02, 0.9)
Churn = np.where(np.random.rand(n) < churn_prob, "Yes", "No")

df = pd.DataFrame({
    "customerID": ["CUST"+str(i).zfill(5) for i in range(n)],
    "gender": gender,
    "SeniorCitizen": SeniorCitizen,
    "Partner": Partner,
    "Dependents": Dependents,
    "tenure": tenure,
    "PhoneService": PhoneService,
    "MultipleLines": MultipleLines,
    "InternetService": InternetService,
    "OnlineSecurity": OnlineSecurity,
    "OnlineBackup": OnlineBackup,
    "DeviceProtection": DeviceProtection,
    "TechSupport": TechSupport,
    "StreamingTV": StreamingTV,
    "StreamingMovies": StreamingMovies,
    "Contract": Contract,
    "PaperlessBilling": PaperlessBilling,
    "PaymentMethod": PaymentMethod,
    "MonthlyCharges": MonthlyCharges,
    "TotalCharges": TotalCharges,
    "Churn": Churn
})
df.to_csv("customer_churn.csv", index=False)
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,CUST00000,Female,0,No,No,38,Yes,Yes,DSL,No,...,No,Yes,No,Yes,Month-to-month,No,Electronic check,66.88,2540.31,Yes
1,CUST00001,Male,0,Yes,No,66,Yes,No,DSL,No,...,No,No internet service,No,No,Month-to-month,No,Mailed check,69.23,4571.80,No
2,CUST00002,Male,0,No,No,4,No,Yes,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Credit card (automatic),98.06,318.25,Yes
3,CUST00003,Male,0,No,No,64,Yes,No,DSL,No internet service,...,No,No internet service,No,No internet service,One year,No,Electronic check,55.48,3611.43,No
4,CUST00004,Female,0,Yes,No,14,Yes,No,Fiber optic,No,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,65.58,947.72,Yes


In [4]:
# Cell 2 - preprocessing and model training
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

X = df.drop(columns=["customerID","Churn"])
y = (df["Churn"]=="Yes").astype(int)

cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(include=['number']).columns.tolist()

preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
])

# Try LightGBM/XGBoost if available (optional). Otherwise use RandomForest.
model = None
try:
    from xgboost import XGBClassifier
    model = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=5, use_label_encoder=False, eval_metric='logloss')
    model_name = "XGBoost"
except Exception:
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model_name = "RandomForest"

pipe = Pipeline([("pre", preprocessor), ("clf", model)])
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:,1] if hasattr(pipe.named_steps['clf'], "predict_proba") else pipe.decision_function(X_test)

auc = roc_auc_score(y_test, y_proba)
print("Model:", model_name)
print("AUC:", round(auc,4))
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:28:07] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model: XGBoost
AUC: 0.6893
              precision    recall  f1-score   support

           0       0.74      0.84      0.79       410
           1       0.52      0.36      0.43       190

    accuracy                           0.69       600
   macro avg       0.63      0.60      0.61       600
weighted avg       0.67      0.69      0.67       600



In [5]:
# Cell 3 - save reports
report = f"""Model report - Interpretable ML: SHAP Churn Project
Model used: {model_name}
AUC: {auc:.4f}

Hyperparameters (estimator):
{pipe.named_steps['clf'].get_params() if hasattr(pipe.named_steps['clf'], 'get_params') else 'N/A'}

Classification report:
"""
from sklearn.metrics import classification_report
report += classification_report(y_test, y_pred)

with open("model_report.txt","w") as f:
    f.write(report)

exec_sum = f"""Executive summary - Customer Churn SHAP Analysis

Model: {model_name}
AUC: {auc:.3f}

Top drivers (expected):
- Contract: Month-to-month increases churn
- Short tenure (<12 months)
- Fiber optic customers
- Paperless billing

Recommendations:
1. Early-retention offers for new customers.
2. Move Month-to-month customers to longer contracts via incentives.
3. Target Fiber customers with retention campaigns.
"""
with open("executive_summary.txt","w") as f:
    f.write(exec_sum)

print("Reports saved: model_report.txt, executive_summary.txt")


Reports saved: model_report.txt, executive_summary.txt


# Cell 3 - save reports
report = f"""Model report - Interpretable ML: SHAP Churn Project
Model used: {model_name}
AUC: {auc:.4f}

Hyperparameters (estimator):
{pipe.named_steps['clf'].get_params() if hasattr(pipe.named_steps['clf'], 'get_params') else 'N/A'}

Classification report:
"""
from sklearn.metrics import classification_report
report += classification_report(y_test, y_pred)

with open("model_report.txt","w") as f:
    f.write(report)

exec_sum = f"""Executive summary - Customer Churn SHAP Analysis

Model: {model_name}
AUC: {auc:.3f}

Top drivers (expected):
- Contract: Month-to-month increases churn
- Short tenure (<12 months)
- Fiber optic customers
- Paperless billing

Recommendations:
1. Early-retention offers for new customers.
2. Move Month-to-month customers to longer contracts via incentives.
3. Target Fiber customers with retention campaigns.
"""
with open("executive_summary.txt","w") as f:
    f.write(exec_sum)

print("Reports saved: model_report.txt, executive_summary.txt")


# <font color='blue'>Why Learn Python?</font>

---

* Is a free and open-source.
* Is a high-level and interpreted general purpose programming language.
* Its simple syntax makes it suitable for learning programming as a first language.
* Has a very extensive standard library and over 150,000 external libraries.
* Is portable and extensible. It interoprate with other languages such as Fortran, C, C++, Java,...
* Has a large community.

**Benefits of Learning Python**

![fig_python](https://static.javatpoint.com/interview/images/advantages-of-python.png)
Image Source: www.javatpoint.com

Python finds applications in areas such as:

+ Web Programming
+ Data Science
+ Machine Learning and Artificial Intelligence
+ Scripting & Automation
+ Games
+ Desktop GUI applications
+ Database access
          
![fig_python](https://hackernoon.com/hn-images/1*jVKTE1dd8CVv4WEtcidCPA.png)

# <font color='blue'> Python Versions</font>

----

Python has two major versions:

+ **2.x**: Released in 2000. The latest version is 2.7 released in 2010. __It is not recommended for use in new projects__.
+ **3.x**: Released in 2008 to fix problems that exist in Python 2.x.
   - The nature of these changes is such that Python 3 was incompatible with Python 2. It is backward incompatible.
   - Python 3 isn’t compatible with Python 2.
   - You should use the latest versions of Python 3 for your new projects.

**For this class, we will use Python 3.x.**

# <font color='blue'>Ways to Run Python</font>

---
We want to introduce four ways to run Python:

* Interactive Command Line through `python`
* Interactive Command Line through `ipython`
* Scripting
* Jupyter Notebook


You first need to make sure that Python is install on your system.
With Linux/Unix systems, a default Python interpreter comes with the operating system. To test this, from the terminal type:

`which python`

and you will see something like:

`/usr/bin/python`


### <font color='red'> Interactive Command Line through `python` </font>

---

* You can use the interpreter in the interactive mode to test some commands.
* You need to type `python` from the command line.
* You can directly type in Python code, and press `Enter` to get the output.
* You can exit the interactive mode with `quit()` or `exit` command or `^Ctrl + D`.
* The sequence you enter will not be saved if you close the current session.

**Sample Session**

1. Open your terminal/Anaconda prompt and type:

`python`


2. From this interactive shell, you will notice that the prompt is `>>>`. This is now allowing us to type Python code directly and execute it.

3. Now type:

`print('Hello world!')`

4. You should see the output on the screen.

5. To exit the Python shell type `exit()` and press `Return` key.



### <font color='red'>Interactive Command Line through `ipython` </font>

---
* IPython is an interactive shell for the Python programming language that offers enhanced introspection, additional shell syntax, tab completion and rich history.
* It does not come by default with Python.
* IPython gives you all that you get in the basic interpreter but with a lot extra (line numbers, advanced editing, more functions, help functions etc).
* It can be started by typing `ipython` at the command line.
* The main aesthetic difference between the Python interpreter and the enhanced IPython interpreter lies in the command prompt: Python uses `>>>` by default, while IPython uses numbered commands (e.g. `In [1]:`).

**Sample Session**

1. Open your terminal/Anaconda prompt and type:

`ipython`

2. This is an enhanced interactive shell that has many features (tab-completion, woot!). It also has a prompt that is numbered.

3. Now type:

`print('Hello world!')`

4. You should see the output on the screen.

5. To exit the Python shell type `exit()` and press `Return` key.

### <font color='red'> Scripting </font>

---
* Real Python programs are made as scripts and look like simple text files.
* These files are given extensions `.py`.
* You can create text files using whatever text editor you like.
* To run the script you need to use the programming language interpreter and specify the name of the created file as an additional parameter:

```
       %python my_python_file.py
```

**Sample Session**

1. Start a new ASCII/text document named `helloworld.py` and enter the following text:

`print('Hello world!')`

2. Save the document and then in your terminal/Anaconda prompt, go to the directory containing the newlt created file.


3. From the terminal/Anaconda prompt, type:

`python helloworld.py`

4. You should see the output on the screen.

### <font color='red'> Jupyter Notebook </font>

---
* A useful hybrid of the interactive terminal and the self-contained script is the Jupyter notebook, a document format that allows executable code, formatted text, graphics, and even interactive features to be combined into a single document.
* Though the notebook began as a Python-only format, it has since been made compatible with a large number of programming languages.
* The notebook is useful both as a development environment, and as a means of sharing work via rich computational and data-driven narratives that mix together code, figures, data, and text.


**Sample Session**

1. Open your terminal/Anaconda Prompt and type:

`jupyter notebook`

2. This directs you to a web browser and you can navigate to an already existing notebook or create one (right side menu New -> Python 3).

3. This will open up a new Untitled notebook where you can directly input Python code, Markup formatted text, or have raw text.

4. Now type:

`print('Hello world!')`

5. Press `Shift+Enter`, `Cntrl+Enter` or click `Cells -> Run Cells` or use the `Play` button near the top of the page.

6. You should see the output on the screen.

7. Exit via closing the browser windows and stopping the server running in the terminal/command prompt (most likely with a Cntrl+C).

## <font color='red'>Other Ways</font>

* [10 Best Python IDE & Code Editors](https://hackr.io/blog/best-python-ide)
* [Python IDEs and Code Editors (Guide)](https://realpython.com/python-ides-code-editors-guide/)
* [Google Colaboratory](http://colab.research.google.com)
* [Binder](http://mybinder.org)
* [Microsoft Azure](http://notebooks.azure.com)

# <font color='blue'>References</font>

* [How to Run Python Code](https://jakevdp.github.io/WhirlwindTourOfPython/01-how-to-run-python-code.html)
* [How To Run Your Python Scripts](https://www.knowledgehut.com/blog/programming/run-python-scripts)
* [Getting Started with Python in Visual Studio Code](https://code.visualstudio.com/docs/python/python-tutorial)
* [How to Run a Python Script on Mac](https://www.maketecheasier.com/run-python-script-in-mac/)
* [Running Python Scripts from anywhere under Windows](https://correlated.kayako.com/article/40-running-python-scripts-from-anywhere-under-windows)